In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=6a83a9ecc59cda4df4ac04e3b694ccaaa71a60eee70bbc31050e7bbdcd2f0429
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# https://drive.google.com/file/d/1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO/view?usp=drive_link

id='1jU38cWxHrKGLg1nWO_wW8031z7d1HpnO'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('googleplaystore.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('googleplaystore.csv', header=True, inferSchema=True)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Desde los ejercicios de pandas sabemos que hay una linea con Rating 19, lo cual no es posible, la sacamos

In [ ]:
rdd = df.rdd.filter(lambda x: x.App != "Life Made WI-Fi Touchscreen Photo Frame").cache()

Vemos un poco como se presentan los datos del registro

In [ ]:
rdd.take(5)

[Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up'),
 Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up'),
 Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver='1.2.4', Android Ver='4.0.3 and up'),
 Row(App='Sketch - Draw & Paint', Category='ART_AND_DESIGN', Rating=4.5, Reviews='215644', Size='25M

Vemos que hay apps repetidas

In [ ]:
aux = rdd.map(lambda x: (x[0], 1)).reduceByKey(lambda x, y: x + y)
aux1 = aux.filter(lambda x: x[1] > 1)
aux1.count()

798

En mi criterio las sacamos

In [ ]:
rdd1 = rdd.map(lambda x: (x[0], x)).reduceByKey(lambda x, y: x)
rdd1.take(5)

[('Photo Editor & Candy Camera & Grid & ScrapBook',
  Row(App='Photo Editor & Candy Camera & Grid & ScrapBook', Category='ART_AND_DESIGN', Rating=4.1, Reviews='159', Size='19M', Installs='10,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='January 7, 2018', Current Ver='1.0.0', Android Ver='4.0.3 and up')),
 ('Coloring book moana',
  Row(App='Coloring book moana', Category='ART_AND_DESIGN', Rating=3.9, Reviews='967', Size='14M', Installs='500,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design;Pretend Play', Last Updated='January 15, 2018', Current Ver='2.0.0', Android Ver='4.0.3 and up')),
 ('U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  Row(App='U Launcher Lite – FREE Live Cool Themes, Hide Apps', Category='ART_AND_DESIGN', Rating=4.7, Reviews='87510', Size='8.7M', Installs='5,000,000+', Type='Free', Price='0', Content Rating='Everyone', Genres='Art & Design', Last Updated='August 1, 2018', Current Ver=

Nos quedamos unicamente con aquellas apps que en el nombre tengan la palabra FREE. Cacheamos ya que hacemos dos operaciones

In [ ]:
rdd2 = rdd1.filter(lambda x: "FREE" in x[0]).cache()
rdd2.count()

43

Nos quedamos unicamente con el nombre de la app y su rating. Hacemos collect porque sabemos que son pocas

In [ ]:
rdd3 = rdd2.map(lambda x: (x[0], x[1][2]))
rdd3.collect()

[('U Launcher Lite – FREE Live Cool Themes, Hide Apps', 4.7),
 ('Quick PDF Scanner + OCR FREE', 4.2),
 ('FREE VIDEO CHAT - LIVE VIDEO AND TEXT CHAT', 4.8),
 ('FREE LIVE TALK', 4.9),
 ('FREE Stock Market Trading Tips', 3.9),
 ('Cookpad - FREE recipe search makes fun cooking · musical making!', 4.1),
 ('DELISH KITCHEN - FREE recipe movies make food fun and easy!', 4.6),
 ('Plants vs. Zombies FREE', 4.4),
 ('Cut the Rope FULL FREE', 4.4),
 ('All-in-One Mahjong 3 FREE', 4.5),
 ('Sounds for Toddlers FREE', 4.4),
 ('Camera ZOOM FX - FREE', 4.0),
 ('FREEDOME VPN Unlimited anonymous Wifi Security', 4.3),
 ('Northern Lights FREE (Aurora)', 4.1),
 ('AndroZip™ FREE File Manager', 4.2),
 ('tinyCam Monitor FREE', 4.0),
 ('AC Remote for O General - NOW FREE', 3.7),
 ('Remote For ATT U-verse TV - NOW FREE', 2.5),
 ('30 Day Ab Challenge FREE', 4.3),
 ("Ladies' Ab Workout FREE", 4.0),
 ('AC Remote for Haier - NOW FREE', 3.9),
 ('AC Remote for Daikin - NOW FREE', 3.5),
 ('AC Remote for LG - NOW FREE', 3

Vimos que hay ratings con nans, los sacamos

In [ ]:
rdd4 = rdd3.filter(lambda x: not isinstance(x[1], float) or not math.isnan(x[1]))
rdd4.collect()

[('U Launcher Lite – FREE Live Cool Themes, Hide Apps', 4.7),
 ('Quick PDF Scanner + OCR FREE', 4.2),
 ('FREE VIDEO CHAT - LIVE VIDEO AND TEXT CHAT', 4.8),
 ('FREE LIVE TALK', 4.9),
 ('FREE Stock Market Trading Tips', 3.9),
 ('Cookpad - FREE recipe search makes fun cooking · musical making!', 4.1),
 ('DELISH KITCHEN - FREE recipe movies make food fun and easy!', 4.6),
 ('Plants vs. Zombies FREE', 4.4),
 ('Cut the Rope FULL FREE', 4.4),
 ('All-in-One Mahjong 3 FREE', 4.5),
 ('Sounds for Toddlers FREE', 4.4),
 ('Camera ZOOM FX - FREE', 4.0),
 ('FREEDOME VPN Unlimited anonymous Wifi Security', 4.3),
 ('Northern Lights FREE (Aurora)', 4.1),
 ('AndroZip™ FREE File Manager', 4.2),
 ('tinyCam Monitor FREE', 4.0),
 ('AC Remote for O General - NOW FREE', 3.7),
 ('Remote For ATT U-verse TV - NOW FREE', 2.5),
 ('30 Day Ab Challenge FREE', 4.3),
 ("Ladies' Ab Workout FREE", 4.0),
 ('AC Remote for Haier - NOW FREE', 3.9),
 ('AC Remote for Daikin - NOW FREE', 3.5),
 ('AC Remote for LG - NOW FREE', 3

Por ultimo nos quedamos con la de peor rating

In [ ]:
rdd5 = rdd4.reduce(lambda a, b: a if a[1] < b[1] else b)
rdd5

('Remote For ATT U-verse TV - NOW FREE', 2.5)